In [7]:
import pandas as pd
import os
from openai import OpenAI

In [8]:
current_directory = os.getcwd()

parent_directory = os.path.dirname(current_directory)

DATA_PATH = os.path.join(parent_directory, "data")
DATA_TOURNAMENT = os.path.join(DATA_PATH, "tournament_hints_data.parquet")

DATA_TOURNAMENT = os.path.join(DATA_PATH, "tournament_hints_data.parquet")
DATA_BUSINESS = os.path.join(DATA_PATH, "Business category taxonomy.xlsx")
DATA_NAICS = os.path.join(
    DATA_PATH,
    "naics3_tournament_taxonomy.xlsx",
)

openai_api_key = "sk-proj-OSCk0tPAJxvpKtxmZIObT3BlbkFJA9hXFC8sq06DAxMmJgQl"

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

Current Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament/marian
Parent Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament


In [9]:
df_naics = pd.read_excel(DATA_NAICS)
df_business = pd.read_excel(DATA_BUSINESS)

In [10]:
client = OpenAI(api_key=openai_api_key)

In [27]:
from tqdm import tqdm

In [39]:
def find_naics_category(description, naics_df):
    responses = []

    # print(f"Try to function {description} with {label}")

    for label, label_description in tqdm(zip(
        naics_df["naics_label"], naics_df["description"]),total=len(naics_df)):
        prompt = f"Given` the description: '{description}',give the probability which match the category: '{label}' which is described as: '{label_description}'. Compute and return a probability score from 0 to 1. Return just the probability score."

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "user", "content": prompt},
                ],
                max_tokens=90,
                temperature=0,
            )

            responses.append((label, response.choices[0].message.content.strip()))

        except Exception as e:
            print(f"An error occurred: {e}")

    result = sorted(responses, key=lambda x: x[1], reverse=True)[0][0]

    print(f"{result} ----> {description}")

    return result


# Mapping each business label to a NAICS category
df_business["Mapped NAICS Label"] = df_business["description"].apply(
    lambda desc: find_naics_category(desc, df_naics)
)

# Saving the mapped labels
df_business.to_csv("mapped_business_labels.csv", index=False)

100%|██████████| 96/96 [01:25<00:00,  1.12it/s]


Motor Vehicle and Parts Dealers ----> Businesses categorized under ATVs Dealers & Services specialize in the sale, maintenance, and rental of all-terrain vehicles (ATVs). These establishments offer a range of services related to ATVs, including sales of new and used vehicles, repair and maintenance services, as well as rental options for recreational use. Customers can expect expert guidance on ATV selection, repair services to ensure optimal performance, and convenient rental options for outdoor adventures. The industry is dedicated to meeting the diverse needs of ATV enthusiasts, providing a one-stop destination for all ATV-related requirements.


100%|██████████| 96/96 [01:18<00:00,  1.22it/s]


Ambulatory Health Care Services ----> Abortion Clinics provide medical services related to pregnancy termination in a clinical setting. These facilities offer a range of reproductive health services, including counseling, medical procedures, and follow-up care. Patients seeking abortion services may visit these clinics for confidential and professional healthcare assistance.


100%|██████████| 96/96 [05:30<00:00,  3.44s/it]


...

100%|██████████| 96/96 [01:30<00:00,  1.06it/s]


Educational Services ----> Adult Education Schools provide specialized educational services to adults seeking to further their knowledge and skills beyond the traditional academic setting. These institutions offer a variety of programs tailored to meet the diverse needs of adult learners, including vocational training, technical education, apprenticeship programs, and educational support services. Adult Education Schools play a crucial role in equipping individuals with the necessary tools and expertise to enhance their career prospects and personal development.


100%|██████████| 96/96 [01:23<00:00,  1.14it/s]


Performing Arts, Spectator Sports, and Related Industries ----> Adult Entertainment Clubs typically refer to establishments that offer adult-oriented entertainment services and activities for patrons. These venues often feature live performances, adult-themed shows, and other forms of entertainment catering to mature audiences. Customers visiting adult entertainment clubs seek a unique and specialized entertainment experience that may include adult beverages, socializing, and various forms of adult entertainment. These establishments are distinct from traditional bars or nightclubs due to their focus on adult-themed entertainment offerings.


 29%|██▉       | 28/96 [00:26<01:05,  1.03it/s]

In [ ]:
# Mapping each business label to a NAICS category
df_business["Mapped NAICS Label"] = df_business["description"].apply(
    lambda desc: find_naics_category(desc, df_naics)
)

# Saving the mapped labels
df_business.to_csv("mapped_business_labels.csv", index=False)

In [ ]:
def find_naics_category_second_approach(labels, naics_df):
    responses = []

    # print(f"Try to function {description} with {label}")

    for label, label_description in zip(
        naics_df["naics_label"], naics_df["description"]
    ):
        prompt = f"Given` the description: '{label_description}' with  '{label}',choose one of the following labels: '{list(naics_df["nics"])}' which is described as: '{label_description}'. Compute and return a probability score from 0 to 1. Return just the probability score."

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    # {
                    #     "role": "system",
                    #     "content": "You are tasked with evaluating business descriptions and categorizing them based on a likelihood score. For each provided description and category, compute and return a probability score from 0 to 1.",
                    # },
                    {"role": "user", "content": prompt},
                ],
                max_tokens=90,
                temperature=0,
            )

            responses.append((label, response.choices[0].message.content.strip()))

        except Exception as e:
            print(f"An error occurred: {e}")

    print(responses)

    result = sorted(responses, key=lambda x: x[1], reverse=True)[0][0]

    return result


# Mapping each business label to a NAICS category
df_business["Mapped NAICS Label"] = df_business["description"].apply(
    lambda desc: find_naics_category(desc, df_naics)
)

# Saving the mapped labels
df_business.to_csv("mapped_business_labels.csv", index=False)

In [23]:
test = df_business[:100]

In [24]:
test

,label,description
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...
1,Abortion Clinics,Abortion Clinics provide medical services rela...
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...
...,...,...
95,Butcher shop,A butcher shop is a retail establishment speci...
96,Buyback Shops,Buyback Shops are retail establishments that s...
97,CBD and Medical Marijuana,The CBD and Medical Marijuana category encompa...
98,Cabinet Makers,Cabinet Makers specialize in the skilled craft...
